# Finite State Machines in Digital Design

*From state diagrams to hardware implementation*

---

## Prerequisites

This tutorial assumes you have completed:

- **[D Flip-Flops in Digital Design](d_flipflop_tutorial.ipynb)** — understanding edge-triggered storage and clock signals
- **[Timing Diagrams in Digital Design](timing_diagrams_tutorial.ipynb)** — reading waveforms and understanding signal timing

You should also be familiar with:

- **Binary numbers** and basic Boolean algebra (AND, OR, NOT)
- **Logic gates** and how to combine them into circuits

---

## What is a Finite State Machine?

Finite State Machines (FSMs) are fundamental building blocks in digital design. They model systems that can be in one of a finite number of states, transitioning between states based on inputs.

An FSM consists of:

- **States**: A finite set of conditions the system can be in
- **Inputs**: External signals that influence state transitions
- **Outputs**: Signals produced by the machine
- **Transitions**: Rules for moving between states based on inputs
- **Initial State**: The starting state when the system powers on

Think of a traffic light: it cycles through states (Red → Green → Yellow → Red), with timing inputs controlling transitions and the light color as output.

---

## State Diagrams: Visualizing FSMs

State diagrams use circles for states and arrows for transitions. Let's visualize a simple two-state FSM that toggles between ON and OFF based on a button press.

This is a **Moore machine** — the output (LED on/off) depends only on the current state.

:::{note}
**About the "button" input:** In FSMs, all inputs are **digital signals synchronized to the clock**. The "button" here represents a clean 0/1 signal sampled at each clock edge, not a raw physical button (which would be asynchronous and bouncy). In practice, this signal would come from button debouncing and edge-detection circuitry.
:::

**Color convention:** Green = output HIGH (LED on), Grey = output LOW (LED off)

In [ ]:
from graphviz import Digraph
from IPython.display import display, SVG

dot = Digraph('toggle_fsm')
dot.attr(rankdir='LR')
dot.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='12', width='0.8')
dot.attr('edge', fontname='Arial', fontsize='10')

dot.node('OFF', 'OFF\nLED=0', fillcolor='#6b7280', fontcolor='white')
dot.node('ON', 'ON\nLED=1', fillcolor='#10b981', fontcolor='white')

dot.node('start', '', shape='point', width='0')
dot.edge('start', 'OFF')

dot.edge('OFF', 'ON', label='button=1', color='#3b82f6', fontcolor='#3b82f6', penwidth='2')
dot.edge('ON', 'OFF', label='button=1', color='#3b82f6', fontcolor='#3b82f6', penwidth='2')

dot.edge('OFF', 'OFF', label='button=0', color='#9ca3af', fontcolor='#6b7280')
dot.edge('ON', 'ON', label='button=0', color='#9ca3af', fontcolor='#6b7280')

display(SVG(dot.pipe(format='svg')))

This FSM has:
- **2 states**: OFF, ON
- **1 input**: button (0 or 1)
- **1 output**: LED (0=off, 1=on) — determined by the **current state**
  - State OFF → LED = 0
  - State ON → LED = 1
- **Transitions**: Toggle state when button=1, stay in current state when button=0
- **Initial state**: OFF (indicated by the incoming arrow with no source state)

**Understanding transitions:**
- **button=1** means: "the button signal was HIGH when sampled at the clock edge"
- **button=0** means: "the button signal was LOW when sampled at the clock edge"
- State changes occur synchronously on the rising clock edge

**Note:** This is a Moore machine because the output depends only on which state we're in, not on the input value.

### State Table

State diagrams are visual, but state tables provide a complete, unambiguous specification that maps directly to hardware.

The same FSM can be represented as a table showing all possible transitions:

| Current State | Input (button) | Output (LED) | Next State |
|--------------|----------------|--------------|------------|
| OFF          | 0              | 0            | OFF        |
| OFF          | 1              | 0            | ON         |
| ON           | 0              | 1            | ON         |
| ON           | 1              | 1            | OFF        |

**How to read this table (Moore machine):**

- **Output column**: Shows the LED value *while in the current state* (before the clock edge)
- **Next State column**: Shows where we go *after the clock edge* based on the input

**Example walkthrough:**

- **Row 1**: In state OFF (LED=0). Input button=0. LED outputs 0, stay in OFF.

- **Row 2**: In state OFF (LED=0). Input button=1. LED outputs 0, transition to ON.
  - *After clock edge*: Now in ON state → LED becomes 1

- **Row 3**: In state ON (LED=1). Input button=0. LED outputs 1, stay in ON.

- **Row 4**: In state ON (LED=1). Input button=1. LED outputs 1, transition to OFF.
  - *After clock edge*: Now in OFF state → LED becomes 0

**Key insight:** The output shown is what the current state produces. After the clock edge, you enter the next state and its output becomes active.

---

## Moore vs Mealy Machines

There are two fundamental types of FSMs, differing in how outputs are determined:

| Aspect | Moore | Mealy |
|--------|-------|-------|
| **Output depends on** | Current state only | Current state AND inputs |
| **Output associated with** | States (inside circles) | Transitions (on arrows) |
| **States needed** | More states may be needed | Fewer states possible |
| **Timing complexity** | Simpler | More complex |
| **Output changes** | Only on clock edges (synchronous) | Can change with inputs (asynchronous) |
| **Notation** | State/Output | Input/Output on arrows |
| **Typical use cases** | Controllers, most general-purpose FSMs, CPU control units | Sequence detectors, protocol handlers, designs requiring minimal states |
| **Adoption in practice** | **Most common** — easier to design, debug, and verify; preferred for synthesis tools | Used when state minimization is critical or faster response needed |

**Why Moore is more common:**
- Outputs are **glitch-free** — they only change on clock edges, avoiding hazards
- **Easier timing analysis** — output timing is predictable and synchronous
- **Simpler verification** — outputs depend only on state, not input combinations
- **Better for synthesis** — HDL tools optimize Moore machines more reliably

**When to use Mealy:**
- Need **one clock cycle faster** response (output changes immediately with input)
- **State count matters** (limited flip-flops, or very complex state space)
- Designing **protocol interfaces** where fast acknowledgment is critical

(sequence-detector-example)=
### Example: Sequence Detector

Let's visualize both for the same problem: **detecting when input X has been 1 for two consecutive clock cycles**. Output Z goes HIGH when we've seen two 1s in a row.

**Color convention:** Blue = pattern not found yet (Z=0), Green = two consecutive 1s detected (Z=1)

In [ ]:
from graphviz import Digraph
from IPython.display import display, HTML

# === MOORE MACHINE ===
moore = Digraph('moore')
moore.attr(rankdir='LR', label='Moore Machine\n(Output depends on state only)\n3 states needed', labelloc='t', fontsize='14')
moore.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='11', width='0.7')
moore.attr('edge', fontname='Arial', fontsize='9')

moore.node('S0', 'S0\nZ=0', fillcolor='#3b82f6', fontcolor='white')
moore.node('S1', 'S1\nZ=0', fillcolor='#3b82f6', fontcolor='white')
moore.node('S2', 'S2\nZ=1', fillcolor='#10b981', fontcolor='white')
moore.node('start_m', '', shape='point', width='0')
moore.edge('start_m', 'S0')
moore.edge('S0', 'S1', label='X=1', penwidth='1.5')
moore.edge('S1', 'S2', label='X=1', penwidth='1.5')
moore.edge('S2', 'S2', label='X=1', penwidth='1.5')
moore.edge('S0', 'S0', label='X=0', color='#9ca3af', fontcolor='#6b7280')
moore.edge('S1', 'S0', label='X=0', color='#9ca3af', fontcolor='#6b7280')
moore.edge('S2', 'S0', label='X=0', color='#9ca3af', fontcolor='#6b7280')

# === MEALY MACHINE ===
mealy = Digraph('mealy')
mealy.attr(rankdir='LR', label='Mealy Machine\n(Output depends on state AND input)\n2 states needed', labelloc='t', fontsize='14')
mealy.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='11', width='0.7')
mealy.attr('edge', fontname='Arial', fontsize='9')

mealy.node('M0', 'S0', fillcolor='#3b82f6', fontcolor='white')
mealy.node('M1', 'S1', fillcolor='#3b82f6', fontcolor='white')
mealy.node('start_e', '', shape='point', width='0')
mealy.edge('start_e', 'M0')
mealy.edge('M0', 'M1', label='X=1 / Z=0', penwidth='1.5')
mealy.edge('M1', 'M1', label='X=1 / Z=1', color='#10b981', fontcolor='#10b981', penwidth='2')
mealy.edge('M0', 'M0', label='X=0 / Z=0', color='#9ca3af', fontcolor='#6b7280')
mealy.edge('M1', 'M0', label='X=0 / Z=0', color='#9ca3af', fontcolor='#6b7280')

moore_svg = moore.pipe(format='svg').decode('utf-8')
mealy_svg = mealy.pipe(format='svg').decode('utf-8')

html = f'''
<div style="display: flex; justify-content: space-around; flex-wrap: wrap; gap: 20px; align-items: flex-start;">
    <div>{moore_svg}</div>
    <div>{mealy_svg}</div>
</div>
'''
display(HTML(html))

**Key observation from this example:**

Notice how the Mealy machine achieves the same detection with only **2 states** compared to Moore's **3 states**. This is because Mealy's output depends on both state and input, allowing more information to be encoded per state. However, this comes at the cost of more complex timing analysis since outputs can change asynchronously with inputs.

#### State Tables

Now let's look at the formal state table representations for both Moore and Mealy versions of our sequence detector.

##### Moore Machine State Table

For the sequence detector Moore machine:

| Current State | Input (X) | Output (Z) | Next State |
|--------------|-----------|------------|------------|
| S0           | 0         | 0          | S0         |
| S0           | 1         | 0          | S1         |
| S1           | 0         | 0          | S0         |
| S1           | 1         | 0          | S2         |
| S2           | 0         | 1          | S0         |
| S2           | 1         | 1          | S2         |

This table follows the Moore machine format: outputs depend only on the current state, and state transitions occur on clock edges.

##### Mealy Machine State Table

For the Mealy version:

| Current State | X=0 (Next/Out) | X=1 (Next/Out) |
|--------------|----------------|----------------|
| S0           | S0/0           | S1/0           |
| S1           | S0/0           | S1/1           |

**How to read this table (Mealy machine):**

- **Notation**: "NextState/Output" shows both next state and output for that input
- **Output timing**: Output is produced *combinationally* (changes as soon as input changes, independent of the clock)
- **Key difference**: Output can change *before* the clock edge when input changes (asynchronous to state transitions)

**Example:** Row 2, column "X=1" shows "S1/1" means: currently in S1 with input X=1, output Z=1 is produced **combinationally** (immediately when X becomes 1, not waiting for a clock edge), and we'll stay in S1 after the next clock edge.

**Moore vs Mealy timing:**
- **Moore**: Output changes only on clock edges (when state changes) — synchronous
- **Mealy**: Output changes immediately when input changes (combinational logic) — can be asynchronous to clock

---

## Implementing the Sequence Detector in Hardware

Now let's walk through how to translate our [sequence detector state diagram](#sequence-detector-example) into actual digital hardware. This involves two main steps:

1. **State Encoding** — Assign binary codes to each state
2. **Derive Boolean Equations** — Use Karnaugh maps to find minimal logic expressions

We'll use the 3-state Moore machine from the sequence detector example above.

### Step 1: State Encoding

To implement an FSM in hardware, we need to assign binary codes to states. The number of flip-flops required is:

$$\text{Number of flip-flops} = \lceil \log_2(\text{number of states}) \rceil$$

For our 3-state Moore machine: $\lceil \log_2(3) \rceil = 2$ flip-flops.

**Why?** 2 flip-flops can represent $2^2 = 4$ distinct states, which is the minimum needed for 3 states. With only 1 flip-flop we'd have $2^1 = 2$ states (insufficient). The 4th state (binary 11) will be unused in our design—we call this a "don't care" state.

#### Common Encoding Schemes

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)
logging.getLogger('matplotlib.font_manager').disabled = True

import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
import logging

logging.getLogger('matplotlib.font_manager').disabled = True
plt.rcParams.update({"text.usetex": False, "mathtext.fontset": "cm"})

fig, ax = plt.subplots(figsize=(12, 5))
ax.axis('off')
ax.set_xlim(0, 100)
ax.set_ylim(0, 60)
ax.set_title('State Encoding Schemes for 4 States', fontsize=16, fontweight='bold', pad=15)

# Binary encoding
binary_box = FancyBboxPatch((5, 25), 25, 30, boxstyle='round,pad=0.02',
                            facecolor='#dbeafe', edgecolor='#3b82f6', linewidth=2)
ax.add_patch(binary_box)
ax.text(17.5, 52, 'Binary', ha='center', fontsize=16, fontweight='bold', color='#1e40af')
ax.text(17.5, 45, 'S0 = 00', ha='center', fontsize=14, family='monospace')
ax.text(17.5, 40, 'S1 = 01', ha='center', fontsize=14, family='monospace')
ax.text(17.5, 35, 'S2 = 10', ha='center', fontsize=14, family='monospace')
ax.text(17.5, 30, 'S3 = 11', ha='center', fontsize=14, family='monospace')
ax.text(17.5, 22, '2 flip-flops', ha='center', fontsize=13, color='#6b7280')

# Gray code
gray_box = FancyBboxPatch((37, 25), 25, 30, boxstyle='round,pad=0.02',
                          facecolor='#d1fae5', edgecolor='#10b981', linewidth=2)
ax.add_patch(gray_box)
ax.text(49.5, 52, 'Gray Code', ha='center', fontsize=16, fontweight='bold', color='#065f46')
ax.text(49.5, 45, 'S0 = 00', ha='center', fontsize=14, family='monospace')
ax.text(49.5, 40, 'S1 = 01', ha='center', fontsize=14, family='monospace')
ax.text(49.5, 35, 'S2 = 11', ha='center', fontsize=14, family='monospace')
ax.text(49.5, 30, 'S3 = 10', ha='center', fontsize=14, family='monospace')
ax.text(49.5, 22, '2 flip-flops', ha='center', fontsize=13, color='#6b7280')

# One-hot
onehot_box = FancyBboxPatch((69, 25), 25, 30, boxstyle='round,pad=0.02',
                            facecolor='#fef3c7', edgecolor='#f59e0b', linewidth=2)
ax.add_patch(onehot_box)
ax.text(81.5, 52, 'One-Hot', ha='center', fontsize=16, fontweight='bold', color='#92400e')
ax.text(81.5, 45, 'S0 = 0001', ha='center', fontsize=14, family='monospace')
ax.text(81.5, 40, 'S1 = 0010', ha='center', fontsize=14, family='monospace')
ax.text(81.5, 35, 'S2 = 0100', ha='center', fontsize=14, family='monospace')
ax.text(81.5, 30, 'S3 = 1000', ha='center', fontsize=14, family='monospace')
ax.text(81.5, 22, '4 flip-flops', ha='center', fontsize=13, color='#6b7280')

plt.tight_layout()
plt.show()

**Choosing an encoding:**

- **Binary**: Minimizes flip-flops; good for resource-constrained designs
- **Gray code**: Adjacent states differ by one bit; reduces glitches in outputs
- **One-hot**: One flip-flop per state; simplifies next-state logic, common in FPGAs

### Step 2: Derive Boolean Equations Using Karnaugh Maps

We use Karnaugh maps to derive minimal Boolean expressions for D1, D0, and Z from the **encoded state table above** (Step 1).

#### Understanding K-Map Structure

For each output (D1, D0, or Z), we create a K-map with:
- **Rows**: Q1Q0 combinations in **Gray code order** (00, 01, 11, 10) - adjacent rows differ by only one bit
- **Columns**: X values (0, 1)

Let's see how to fill these K-maps from the state table:

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)
logging.getLogger('matplotlib.font_manager').disabled = True

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch, FancyArrowPatch
import numpy as np

plt.rcParams.update({"text.usetex": False, "mathtext.fontset": "cm"})

# Show empty K-map structure - compact size
fig, ax = plt.subplots(figsize=(5, 4.5))
ax.set_xlim(-1.5, 3.5)
ax.set_ylim(-1, 5.5)
ax.set_aspect('equal')
ax.axis('off')

ax.text(1.0, 5.0, 'K-Map Structure (2 variables + 1 input)', fontsize=12, 
        fontweight='bold', ha='center',
        bbox=dict(boxstyle='round,pad=0.4', facecolor='#f3f4f6', 
                  edgecolor='#374151', linewidth=2))

# Column headers
for j, x_val in enumerate(['X=0', 'X=1']):
    header_box = FancyBboxPatch((j, 4.0), 1, 0.4, boxstyle='round,pad=0.02',
                               facecolor='#dbeafe', edgecolor='#3b82f6', linewidth=2)
    ax.add_patch(header_box)
    ax.text(j + 0.5, 4.2, x_val, ha='center', va='center',
            fontsize=10, fontweight='bold', color='#1e40af')

# Row header label
row_label_box = FancyBboxPatch((-1.2, 4.0), 1.0, 0.4, boxstyle='round,pad=0.02',
                              facecolor='#dbeafe', edgecolor='#3b82f6', linewidth=2)
ax.add_patch(row_label_box)
ax.text(-0.7, 4.2, r'$Q_1Q_0$', ha='center', va='center',
        fontsize=10, fontweight='bold', color='#1e40af')

# Row headers with Gray code annotation
row_labels = ['00', '01', '11', '10']
gray_notes = ['', '(1 bit)', '(1 bit)', '(1 bit)']
for i, (label, note) in enumerate(zip(row_labels, gray_notes)):
    row_box = FancyBboxPatch((-1.2, 3-i), 1.0, 1, boxstyle='round,pad=0.02',
                            facecolor='#dbeafe', edgecolor='#3b82f6', linewidth=2)
    ax.add_patch(row_box)
    ax.text(-0.7, 3.5 - i, label, ha='center', va='center',
            fontsize=10, fontweight='bold', color='#1e40af')
    if note:
        ax.text(-0.7, 3.2 - i, note, ha='center', va='center',
                fontsize=6, style='italic', color='#3b82f6')

# Empty K-map cells
for i in range(4):
    for j in range(2):
        cell_box = Rectangle((j, 3-i), 1, 1, linewidth=2,
                            edgecolor='#6b7280', facecolor='white')
        ax.add_patch(cell_box)
        ax.text(j + 0.5, 3.5 - i, '?', ha='center', va='center',
               fontsize=16, color='#d1d5db', fontweight='bold')

# Add explanation
explanation = """Gray Code Order: Adjacent rows differ by one bit
This simplifies grouping for minimization."""
ax.text(1.0, -0.5, explanation, ha='center', va='top', fontsize=8,
        color='#374151', style='italic',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='#f0f9ff', 
                  edgecolor='#0369a1', linewidth=1.5))

plt.tight_layout()
plt.show()

#### Filling the K-Map for D1 (Next Q1)

To fill the D1 K-map, we look at the **"D1 (Next Q1)" column** in the encoded state table from Step 1.

**Process:**
1. Find a row in the state table (e.g., Q1=0, Q0=1, X=1)
2. Look at its D1 value (D1=1)
3. Put that value in the K-map cell at row "01", column "1"

**Example:** Row with Q1=0, Q0=1, X=1 → D1=1, so K-map cell at row "01", column "1" contains **1**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)
logging.getLogger('matplotlib.font_manager').disabled = True

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch, FancyArrowPatch
import numpy as np

plt.rcParams.update({"text.usetex": False, "mathtext.fontset": "cm"})

fig, ax = plt.subplots(figsize=(14, 7))
ax.set_xlim(0, 14)
ax.set_ylim(-1, 6)
ax.axis('off')

ax.text(7, 5.5, 'Filling K-Map for D₁ from State Table', fontsize=15, 
        fontweight='bold', ha='center',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='#dbeafe', 
                  edgecolor='#3b82f6', linewidth=2))

# State table (simplified, showing just the relevant columns)
headers = [r'$Q_1$', r'$Q_0$', 'X', r'$D_1$']
table_data = [
    ['0', '0', '0', '0'],
    ['0', '0', '1', '0'],
    ['0', '1', '0', '0'],
    ['0', '1', '1', '1'],  # This one highlighted
    ['1', '0', '0', '0'],
    ['1', '0', '1', '1'],  # This one highlighted
]

# Draw state table
x_start, y_start = 0.5, 4.0
col_width = 0.6
for i, header in enumerate(headers):
    hdr_box = FancyBboxPatch((x_start + i*col_width, y_start), col_width, 0.4,
                             boxstyle='round,pad=0.02', facecolor='#fef3c7',
                             edgecolor='#f59e0b', linewidth=1.5)
    ax.add_patch(hdr_box)
    ax.text(x_start + i*col_width + col_width/2, y_start + 0.2, header,
            ha='center', va='center', fontsize=10, fontweight='bold')

# Table rows
for row_idx, row_data in enumerate(table_data):
    y_pos = y_start - 0.5 - (row_idx * 0.4)
    highlight = row_idx in [3, 5]  # Rows with D1=1
    
    for col_idx, val in enumerate(row_data):
        is_d1_col = col_idx == 3
        if highlight and is_d1_col:
            bg_color = '#fde047'
            edge_color = '#ca8a04'
            edge_width = 2
        elif highlight:
            bg_color = '#fef3c7'
            edge_color = '#f59e0b'
            edge_width = 1.5
        else:
            bg_color = 'white'
            edge_color = '#d1d5db'
            edge_width = 1
        
        cell_box = Rectangle((x_start + col_idx*col_width, y_pos), col_width, 0.35,
                            facecolor=bg_color, edgecolor=edge_color, linewidth=edge_width)
        ax.add_patch(cell_box)
        ax.text(x_start + col_idx*col_width + col_width/2, y_pos + 0.175, val,
               ha='center', va='center', fontsize=10,
               fontweight='bold' if highlight else 'normal')

# Draw K-map for D1
kmap_x, kmap_y = 7.5, 3.5
d1_data = [['0', '0'], ['0', '1'], ['-', '-'], ['0', '1']]

# Column headers
for j, x_val in enumerate(['X=0', 'X=1']):
    header_box = FancyBboxPatch((kmap_x + j, kmap_y + 0.5), 1, 0.4,
                               boxstyle='round,pad=0.02', facecolor='#e5e7eb',
                               edgecolor='#6b7280', linewidth=1.5)
    ax.add_patch(header_box)
    ax.text(kmap_x + j + 0.5, kmap_y + 0.7, x_val, ha='center', va='center',
            fontsize=11, fontweight='bold')

# Row header label
row_label_box = FancyBboxPatch((kmap_x - 1.0, kmap_y + 0.5), 0.8, 0.4,
                              boxstyle='round,pad=0.02', facecolor='#e5e7eb',
                              edgecolor='#6b7280', linewidth=1.5)
ax.add_patch(row_label_box)
ax.text(kmap_x - 0.6, kmap_y + 0.7, r'$Q_1Q_0$', ha='center', va='center',
        fontsize=11, fontweight='bold')

# Row headers
row_labels = ['00', '01', '11', '10']
for i, label in enumerate(row_labels):
    row_box = FancyBboxPatch((kmap_x - 1.0, kmap_y - 0.1 - i), 0.8, 1,
                            boxstyle='round,pad=0.02', facecolor='#e5e7eb',
                            edgecolor='#6b7280', linewidth=1.5)
    ax.add_patch(row_box)
    ax.text(kmap_x - 0.6, kmap_y + 0.4 - i, label, ha='center', va='center',
            fontsize=11, fontweight='bold')

# K-map cells
highlight_cells = [(1, 1), (3, 1)]  # Cells we're filling
for i in range(4):
    for j in range(2):
        val = d1_data[i][j]
        is_highlighted = (i, j) in highlight_cells
        
        if is_highlighted:
            bg_color = '#fde047'
            edge_color = '#ca8a04'
            edge_width = 2.5
        elif val == '-':
            bg_color = '#fef3c7'
            edge_color = '#374151'
            edge_width = 1.5
        elif val == '1':
            bg_color = '#dcfce7'
            edge_color = '#374151'
            edge_width = 1.5
        else:
            bg_color = '#f9fafb'
            edge_color = '#d1d5db'
            edge_width = 1
        
        cell_box = Rectangle((kmap_x + j, kmap_y - 0.1 - i), 1, 1,
                            facecolor=bg_color, edgecolor=edge_color,
                            linewidth=edge_width)
        ax.add_patch(cell_box)
        ax.text(kmap_x + j + 0.5, kmap_y + 0.4 - i, val, ha='center', va='center',
               fontsize=14, fontweight='bold' if is_highlighted else 'normal',
               color='#166534' if val == '1' else '#6b7280')

# Draw arrows showing the mapping
arrow1 = FancyArrowPatch((3.5, 2.45), (6.5, 3.4),
                        arrowstyle='->', mutation_scale=20, color='#dc2626',
                        linewidth=2.5, connectionstyle="arc3,rad=0.3")
ax.add_patch(arrow1)
ax.text(5.0, 3.3, 'Q₁Q₀X=011 → D₁=1', fontsize=9, color='#dc2626',
        fontweight='bold', bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                                     edgecolor='#dc2626', linewidth=1.5))

arrow2 = FancyArrowPatch((3.5, 1.65), (6.5, 1.4),
                        arrowstyle='->', mutation_scale=20, color='#dc2626',
                        linewidth=2.5, connectionstyle="arc3,rad=-0.3")
ax.add_patch(arrow2)
ax.text(5.0, 0.8, 'Q₁Q₀X=101 → D₁=1', fontsize=9, color='#dc2626',
        fontweight='bold', bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                                     edgecolor='#dc2626', linewidth=1.5))

# Add note
note = "Repeat this process for all 7 rows of the state table\nto fill all cells in the K-map"
ax.text(7, -0.5, note, ha='center', va='top', fontsize=10, color='#374151',
        style='italic', bbox=dict(boxstyle='round,pad=0.4', facecolor='#f0f9ff',
                                  edgecolor='#0369a1', linewidth=1.5))

plt.tight_layout()
plt.show()

#### Filling the K-Map for D0 (Next Q0)

To fill the D0 K-map, we look at the **"D0 (Next Q0)" column** in the encoded state table.

**Example:** Row with Q1=0, Q0=0, X=1 → D0=1, so K-map cell at row "00", column "1" contains **1**

Notice that only **one cell** has D0=1. Most cells are 0.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)
logging.getLogger('matplotlib.font_manager').disabled = True

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch, FancyArrowPatch
import numpy as np

plt.rcParams.update({"text.usetex": False, "mathtext.fontset": "cm"})

fig, ax = plt.subplots(figsize=(14, 7))
ax.set_xlim(0, 14)
ax.set_ylim(-1, 6)
ax.axis('off')

ax.text(7, 5.5, 'Filling K-Map for D₀ from State Table', fontsize=15, 
        fontweight='bold', ha='center',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='#fef3c7', 
                  edgecolor='#f59e0b', linewidth=2))

# State table (simplified, showing just the relevant columns)
headers = [r'$Q_1$', r'$Q_0$', 'X', r'$D_0$']
table_data = [
    ['0', '0', '0', '0'],
    ['0', '0', '1', '1'],  # Only this one has D0=1
    ['0', '1', '0', '0'],
    ['0', '1', '1', '0'],
    ['1', '0', '0', '0'],
    ['1', '0', '1', '0'],
]

# Draw state table
x_start, y_start = 0.5, 4.0
col_width = 0.6
for i, header in enumerate(headers):
    hdr_box = FancyBboxPatch((x_start + i*col_width, y_start), col_width, 0.4,
                             boxstyle='round,pad=0.02', facecolor='#fef3c7',
                             edgecolor='#f59e0b', linewidth=1.5)
    ax.add_patch(hdr_box)
    ax.text(x_start + i*col_width + col_width/2, y_start + 0.2, header,
            ha='center', va='center', fontsize=10, fontweight='bold')

# Table rows
for row_idx, row_data in enumerate(table_data):
    y_pos = y_start - 0.5 - (row_idx * 0.4)
    highlight = row_idx == 1  # Only row 1 has D0=1
    
    for col_idx, val in enumerate(row_data):
        is_d0_col = col_idx == 3
        if highlight and is_d0_col:
            bg_color = '#fde047'
            edge_color = '#ca8a04'
            edge_width = 2
        elif highlight:
            bg_color = '#fef3c7'
            edge_color = '#f59e0b'
            edge_width = 1.5
        else:
            bg_color = 'white'
            edge_color = '#d1d5db'
            edge_width = 1
        
        cell_box = Rectangle((x_start + col_idx*col_width, y_pos), col_width, 0.35,
                            facecolor=bg_color, edgecolor=edge_color, linewidth=edge_width)
        ax.add_patch(cell_box)
        ax.text(x_start + col_idx*col_width + col_width/2, y_pos + 0.175, val,
               ha='center', va='center', fontsize=10,
               fontweight='bold' if highlight else 'normal')

# Draw K-map for D0
kmap_x, kmap_y = 7.5, 3.5
d0_data = [['0', '1'], ['0', '0'], ['-', '-'], ['0', '0']]

# Column headers
for j, x_val in enumerate(['X=0', 'X=1']):
    header_box = FancyBboxPatch((kmap_x + j, kmap_y + 0.5), 1, 0.4,
                               boxstyle='round,pad=0.02', facecolor='#e5e7eb',
                               edgecolor='#6b7280', linewidth=1.5)
    ax.add_patch(header_box)
    ax.text(kmap_x + j + 0.5, kmap_y + 0.7, x_val, ha='center', va='center',
            fontsize=11, fontweight='bold')

# Row header label
row_label_box = FancyBboxPatch((kmap_x - 1.0, kmap_y + 0.5), 0.8, 0.4,
                              boxstyle='round,pad=0.02', facecolor='#e5e7eb',
                              edgecolor='#6b7280', linewidth=1.5)
ax.add_patch(row_label_box)
ax.text(kmap_x - 0.6, kmap_y + 0.7, r'$Q_1Q_0$', ha='center', va='center',
        fontsize=11, fontweight='bold')

# Row headers
row_labels = ['00', '01', '11', '10']
for i, label in enumerate(row_labels):
    row_box = FancyBboxPatch((kmap_x - 1.0, kmap_y - 0.1 - i), 0.8, 1,
                            boxstyle='round,pad=0.02', facecolor='#e5e7eb',
                            edgecolor='#6b7280', linewidth=1.5)
    ax.add_patch(row_box)
    ax.text(kmap_x - 0.6, kmap_y + 0.4 - i, label, ha='center', va='center',
            fontsize=11, fontweight='bold')

# K-map cells
highlight_cells = [(0, 1)]  # Only one cell
for i in range(4):
    for j in range(2):
        val = d0_data[i][j]
        is_highlighted = (i, j) in highlight_cells
        
        if is_highlighted:
            bg_color = '#fde047'
            edge_color = '#ca8a04'
            edge_width = 2.5
        elif val == '-':
            bg_color = '#fef3c7'
            edge_color = '#374151'
            edge_width = 1.5
        elif val == '1':
            bg_color = '#dcfce7'
            edge_color = '#374151'
            edge_width = 1.5
        else:
            bg_color = '#f9fafb'
            edge_color = '#d1d5db'
            edge_width = 1
        
        cell_box = Rectangle((kmap_x + j, kmap_y - 0.1 - i), 1, 1,
                            facecolor=bg_color, edgecolor=edge_color,
                            linewidth=edge_width)
        ax.add_patch(cell_box)
        ax.text(kmap_x + j + 0.5, kmap_y + 0.4 - i, val, ha='center', va='center',
               fontsize=14, fontweight='bold' if is_highlighted else 'normal',
               color='#166534' if val == '1' else '#6b7280')

# Draw arrow showing the mapping
arrow = FancyArrowPatch((3.5, 3.25), (7.0, 4.0),
                       arrowstyle='->', mutation_scale=20, color='#dc2626',
                       linewidth=2.5, connectionstyle="arc3,rad=0.3")
ax.add_patch(arrow)
ax.text(5.25, 4.2, 'Q₁Q₀X=001 → D₀=1', fontsize=9, color='#dc2626',
        fontweight='bold', bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                                     edgecolor='#dc2626', linewidth=1.5))

# Add note
note = "D₀ = 1 in only one case: when we're in state S0 (00) and input X=1\nThis is the transition from S0 → S1"
ax.text(7, -0.5, note, ha='center', va='top', fontsize=10, color='#374151',
        style='italic', bbox=dict(boxstyle='round,pad=0.4', facecolor='#fffbeb',
                                  edgecolor='#f59e0b', linewidth=1.5))

plt.tight_layout()
plt.show()

#### Filling the K-Map for Z (Output)

To fill the Z K-map, we look at the **"Z" column** in the encoded state table.

**Key insight for Moore machines:** Z depends **only on the current state** (Q1, Q0), NOT on input X. 

**Example:** Rows with Q1=1, Q0=0 → Z=1, so K-map cells at row "10" contain **1** for BOTH X=0 and X=1

This is why the entire row has the same value!

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)
logging.getLogger('matplotlib.font_manager').disabled = True

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch, FancyArrowPatch
import numpy as np

plt.rcParams.update({"text.usetex": False, "mathtext.fontset": "cm"})

fig, ax = plt.subplots(figsize=(14, 7))
ax.set_xlim(0, 14)
ax.set_ylim(-1, 6)
ax.axis('off')

ax.text(7, 5.5, 'Filling K-Map for Z (Output) - Moore Machine', fontsize=15, 
        fontweight='bold', ha='center',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='#fee2e2', 
                  edgecolor='#ef4444', linewidth=2))

# State table (simplified, showing just the relevant columns)
headers = [r'$Q_1$', r'$Q_0$', 'X', 'Z']
table_data = [
    ['0', '0', '0', '0'],
    ['0', '0', '1', '0'],
    ['0', '1', '0', '0'],
    ['0', '1', '1', '0'],
    ['1', '0', '0', '1'],  # Z=1 regardless of X
    ['1', '0', '1', '1'],  # Z=1 regardless of X
]

# Draw state table
x_start, y_start = 0.5, 4.0
col_width = 0.6
for i, header in enumerate(headers):
    hdr_box = FancyBboxPatch((x_start + i*col_width, y_start), col_width, 0.4,
                             boxstyle='round,pad=0.02', facecolor='#fee2e2',
                             edgecolor='#ef4444', linewidth=1.5)
    ax.add_patch(hdr_box)
    ax.text(x_start + i*col_width + col_width/2, y_start + 0.2, header,
            ha='center', va='center', fontsize=10, fontweight='bold')

# Table rows
for row_idx, row_data in enumerate(table_data):
    y_pos = y_start - 0.5 - (row_idx * 0.4)
    highlight = row_idx in [4, 5]  # Both rows with Q1=1, Q0=0
    
    for col_idx, val in enumerate(row_data):
        is_z_col = col_idx == 3
        if highlight and is_z_col:
            bg_color = '#fde047'
            edge_color = '#ca8a04'
            edge_width = 2
        elif highlight:
            bg_color = '#fee2e2'
            edge_color = '#ef4444'
            edge_width = 1.5
        else:
            bg_color = 'white'
            edge_color = '#d1d5db'
            edge_width = 1
        
        cell_box = Rectangle((x_start + col_idx*col_width, y_pos), col_width, 0.35,
                            facecolor=bg_color, edgecolor=edge_color, linewidth=edge_width)
        ax.add_patch(cell_box)
        ax.text(x_start + col_idx*col_width + col_width/2, y_pos + 0.175, val,
               ha='center', va='center', fontsize=10,
               fontweight='bold' if highlight else 'normal')

# Draw K-map for Z
kmap_x, kmap_y = 7.5, 3.5
z_data = [['0', '0'], ['0', '0'], ['-', '-'], ['1', '1']]

# Column headers
for j, x_val in enumerate(['X=0', 'X=1']):
    header_box = FancyBboxPatch((kmap_x + j, kmap_y + 0.5), 1, 0.4,
                               boxstyle='round,pad=0.02', facecolor='#e5e7eb',
                               edgecolor='#6b7280', linewidth=1.5)
    ax.add_patch(header_box)
    ax.text(kmap_x + j + 0.5, kmap_y + 0.7, x_val, ha='center', va='center',
            fontsize=11, fontweight='bold')

# Row header label
row_label_box = FancyBboxPatch((kmap_x - 1.0, kmap_y + 0.5), 0.8, 0.4,
                              boxstyle='round,pad=0.02', facecolor='#e5e7eb',
                              edgecolor='#6b7280', linewidth=1.5)
ax.add_patch(row_label_box)
ax.text(kmap_x - 0.6, kmap_y + 0.7, r'$Q_1Q_0$', ha='center', va='center',
        fontsize=11, fontweight='bold')

# Row headers
row_labels = ['00', '01', '11', '10']
for i, label in enumerate(row_labels):
    row_box = FancyBboxPatch((kmap_x - 1.0, kmap_y - 0.1 - i), 0.8, 1,
                            boxstyle='round,pad=0.02', facecolor='#e5e7eb',
                            edgecolor='#6b7280', linewidth=1.5)
    ax.add_patch(row_box)
    ax.text(kmap_x - 0.6, kmap_y + 0.4 - i, label, ha='center', va='center',
            fontsize=11, fontweight='bold')

# K-map cells
highlight_cells = [(3, 0), (3, 1)]  # Entire row 10
for i in range(4):
    for j in range(2):
        val = z_data[i][j]
        is_highlighted = (i, j) in highlight_cells
        
        if is_highlighted:
            bg_color = '#fde047'
            edge_color = '#ca8a04'
            edge_width = 2.5
        elif val == '-':
            bg_color = '#fef3c7'
            edge_color = '#374151'
            edge_width = 1.5
        elif val == '1':
            bg_color = '#dcfce7'
            edge_color = '#374151'
            edge_width = 1.5
        else:
            bg_color = '#f9fafb'
            edge_color = '#d1d5db'
            edge_width = 1
        
        cell_box = Rectangle((kmap_x + j, kmap_y - 0.1 - i), 1, 1,
                            facecolor=bg_color, edgecolor=edge_color,
                            linewidth=edge_width)
        ax.add_patch(cell_box)
        ax.text(kmap_x + j + 0.5, kmap_y + 0.4 - i, val, ha='center', va='center',
               fontsize=14, fontweight='bold' if is_highlighted else 'normal',
               color='#166534' if val == '1' else '#6b7280')

# Draw arrows showing the mapping
arrow1 = FancyArrowPatch((3.5, 1.65), (6.5, 0.4),
                        arrowstyle='->', mutation_scale=20, color='#dc2626',
                        linewidth=2.5, connectionstyle="arc3,rad=0.3")
ax.add_patch(arrow1)
ax.text(5.0, 1.3, 'Q₁Q₀=10, X=0 → Z=1', fontsize=9, color='#dc2626',
        fontweight='bold', bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                                     edgecolor='#dc2626', linewidth=1.5))

arrow2 = FancyArrowPatch((3.5, 1.25), (7.5, 0.4),
                        arrowstyle='->', mutation_scale=20, color='#dc2626',
                        linewidth=2.5, connectionstyle="arc3,rad=-0.3")
ax.add_patch(arrow2)
ax.text(5.5, 0.5, 'Q₁Q₀=10, X=1 → Z=1', fontsize=9, color='#dc2626',
        fontweight='bold', bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                                     edgecolor='#dc2626', linewidth=1.5))

# Add note
note = "Moore Machine Property: Z depends only on state (Q₁Q₀), not on input X\nWhen Q₁Q₀=10 (state S2), Z=1 for BOTH X values → entire row is 1"
ax.text(7, -0.5, note, ha='center', va='top', fontsize=10, color='#374151',
        style='italic', bbox=dict(boxstyle='round,pad=0.4', facecolor='#fef2f2',
                                  edgecolor='#ef4444', linewidth=1.5))

plt.tight_layout()
plt.show()

#### Grouping Adjacent 1s for Minimization

Once all K-maps are filled, we look for **rectangular groupings** of adjacent 1s. These groupings help us derive minimal Boolean expressions.

**Grouping rules:**
- Group 1s in powers of 2 (1, 2, 4, 8 cells)
- Groups can be rectangular (rows, columns, or blocks)
- Larger groups = simpler equations
- Don't cares ('-') can be included in groups to make them larger

Let's see the complete K-maps with groupings:

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)
logging.getLogger('matplotlib.font_manager').disabled = True

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch
import numpy as np

plt.rcParams.update({"text.usetex": False, "mathtext.fontset": "cm"})

fig, axes = plt.subplots(1, 3, figsize=(20, 8))
fig.suptitle('Karnaugh Maps with Groupings → Minimal Boolean Equations',
             fontsize=18, fontweight='bold', y=0.98)

d1_data = np.array([['0', '0'], ['0', '1'], ['-', '-'], ['0', '1']])
d0_data = np.array([['0', '1'], ['0', '0'], ['-', '-'], ['0', '0']])
z_data = np.array([['0', '0'], ['0', '0'], ['-', '-'], ['1', '1']])

def draw_kmap(ax, data, title, groupings, equation_text):
    ax.set_xlim(-1.2, 3.5)
    ax.set_ylim(-2.8, 5.8)
    ax.set_aspect('equal')
    ax.axis('off')

    # Title with background
    title_box = FancyBboxPatch((0.3, 5.0), 1.8, 0.6, boxstyle='round,pad=0.05',
                               facecolor='#f3f4f6', edgecolor='#374151', linewidth=2)
    ax.add_patch(title_box)
    ax.text(1.2, 5.3, title, ha='center', va='center', fontsize=15, fontweight='bold')

    # Column headers with background boxes
    for j, x_val in enumerate(['X=0', 'X=1']):
        header_box = FancyBboxPatch((j, 4.1), 1, 0.5, boxstyle='round,pad=0.02',
                                   facecolor='#e5e7eb', edgecolor='#6b7280', linewidth=1.5)
        ax.add_patch(header_box)
        ax.text(j + 0.5, 4.35, x_val, ha='center', va='center',
                fontsize=13, fontweight='bold', color='#1f2937')

    # Row header label
    row_label_box = FancyBboxPatch((-1.1, 4.1), 0.9, 0.5, boxstyle='round,pad=0.02',
                                  facecolor='#e5e7eb', edgecolor='#6b7280', linewidth=1.5)
    ax.add_patch(row_label_box)
    ax.text(-0.65, 4.35, r'$Q_1Q_0$', ha='center', va='center',
            fontsize=13, fontweight='bold', color='#1f2937')

    # Draw groupings first (so they appear behind cells)
    for group in groupings:
        x, y, w, h, color, label, eq_label = group
        # Draw filled rectangle for grouping with dashed border
        ax.add_patch(Rectangle((x, y), w, h, linewidth=3.5, edgecolor=color,
                              facecolor=color, alpha=0.2, linestyle='--', zorder=1))
        # Add group label
        if eq_label:
            ax.text(x + w/2, y + h/2, eq_label, ha='center', va='center',
                   fontsize=11, color=color, fontweight='bold', style='italic',
                   bbox=dict(boxstyle='round,pad=0.35', facecolor='white',
                            edgecolor=color, alpha=0.9, linewidth=2), zorder=3)

    # Row headers with background
    for i, label in enumerate(['00', '01', '11', '10']):
        row_box = FancyBboxPatch((-1.1, 3-i), 0.9, 1, boxstyle='round,pad=0.02',
                                facecolor='#e5e7eb', edgecolor='#6b7280', linewidth=1.5)
        ax.add_patch(row_box)
        ax.text(-0.65, 3.5 - i, label, ha='center', va='center',
                fontsize=13, fontweight='bold', color='#1f2937')

    # Draw K-map cells
    for i in range(4):
        for j in range(2):
            val = data[i][j]
            # Cell background color based on value
            if val == '1':
                cell_color = '#dcfce7'  # Light green for 1s
                text_color = '#166534'
                edge_width = 2.5
            elif val == '-':
                cell_color = '#fef3c7'  # Light yellow for don't cares
                text_color = '#92400e'
                edge_width = 2
            else:
                cell_color = '#f9fafb'  # Very light gray for 0s
                text_color = '#9ca3af'
                edge_width = 1.5

            ax.add_patch(Rectangle((j, 3-i), 1, 1, linewidth=edge_width,
                                  edgecolor='#374151', facecolor=cell_color, zorder=2))
            ax.text(j + 0.5, 3.5 - i, str(val), ha='center', va='center',
                   fontsize=17, fontweight='bold', color=text_color, zorder=4)

    # Add instruction text
    ax.text(1.0, -0.7, 'Group adjacent 1s to minimize:', ha='center',
            fontsize=11, color='#6b7280', style='italic')
    
    # Equation box at bottom
    eq_box = FancyBboxPatch((-0.9, -2.5), 4.0, 1.1, boxstyle='round,pad=0.15',
                           facecolor='#f0f9ff', edgecolor='#0369a1', linewidth=2.5)
    ax.add_patch(eq_box)
    ax.text(1.1, -1.95, equation_text, ha='center', va='center',
            fontsize=13, color='#0c4a6e', fontweight='bold')

# D1 K-map with two overlapping groups
groupings_d1 = [
    (1, 1, 1, 1, '#3b82f6', 'Group 1', r'$Q_0 \cdot X$'),  # Blue: vertical group at X=1, Q0=1
    (1, 0, 1, 1, '#10b981', 'Group 2', r'$Q_1 \cdot X$')   # Green: vertical group at X=1, Q1=1
]
draw_kmap(axes[0], d1_data, r'$D_1$ (Next $Q_1$)', groupings_d1,
          r'$D_1 = Q_0 \cdot X + Q_1 \cdot X$' + '\n' + r'$= X(Q_0 + Q_1)$')

# D0 K-map with one cell (can't group further)
groupings_d0 = [
    (1, 3, 1, 1, '#f59e0b', 'Single cell', r'$\overline{Q_1}\,\overline{Q_0}\,X$')
]
draw_kmap(axes[1], d0_data, r'$D_0$ (Next $Q_0$)', groupings_d0,
          r'$D_0 = \overline{Q_1} \cdot \overline{Q_0} \cdot X$')

# Z K-map with horizontal group spanning both columns
groupings_z = [
    (0, 0, 2, 1, '#ef4444', 'Full row', r'$Q_1$')
]
draw_kmap(axes[2], z_data, r'$Z$ (Output)', groupings_z,
          r'$Z = Q_1$' + '\n(Moore: output depends only on state)')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

**Final Boolean Equations:**

$$D_1 = Q_0 \cdot X + Q_1 \cdot X = X \cdot (Q_0 + Q_1)$$

$$D_0 = \overline{Q_1} \cdot \overline{Q_0} \cdot X$$

$$Z = Q_1$$

These equations can be implemented with AND, OR, and NOT gates to create the next-state and output logic for our FSM.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)
logging.getLogger('matplotlib.font_manager').disabled = True

import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch

plt.rcParams.update({"text.usetex": False, "mathtext.fontset": "cm"})

fig, ax = plt.subplots(figsize=(14, 8))
ax.axis('off')
ax.set_xlim(0, 140)
ax.set_ylim(0, 90)
ax.set_title('FSM Hardware Architecture', fontsize=18, fontweight='bold', pad=15)

ff_color = '#3b82f6'
logic_color = '#10b981'
wire_color = '#374151'

ns_box = FancyBboxPatch((10, 30), 30, 35, boxstyle='round,pad=0.02',
                        facecolor='#d1fae5', edgecolor=logic_color, linewidth=2)
ax.add_patch(ns_box)
ax.text(25, 60, 'Next-State Logic', ha='center', fontsize=14, fontweight='bold', color='#065f46')
ax.text(25, 45, r'$D_1 = X(Q_0+Q_1)$', ha='center', fontsize=12, color='#065f46')
ax.text(25, 39, r'$D_0 = X \cdot \overline{Q_1} \cdot \overline{Q_0}$', ha='center', fontsize=12, color='#065f46')

ff1_box = FancyBboxPatch((55, 50), 20, 18, boxstyle='round,pad=0.02', facecolor='#dbeafe', edgecolor=ff_color, linewidth=2)
ax.add_patch(ff1_box)
ax.text(65, 62, 'D FF', ha='center', fontsize=13, fontweight='bold', color='#1e40af')
ax.text(65, 56, r'$Q_1$', ha='center', fontsize=14, color='#1e40af')

ff0_box = FancyBboxPatch((55, 25), 20, 18, boxstyle='round,pad=0.02', facecolor='#dbeafe', edgecolor=ff_color, linewidth=2)
ax.add_patch(ff0_box)
ax.text(65, 37, 'D FF', ha='center', fontsize=13, fontweight='bold', color='#1e40af')
ax.text(65, 31, r'$Q_0$', ha='center', fontsize=14, color='#1e40af')

out_box = FancyBboxPatch((95, 40), 25, 20, boxstyle='round,pad=0.02', facecolor='#fef3c7', edgecolor='#f59e0b', linewidth=2)
ax.add_patch(out_box)
ax.text(107.5, 55, 'Output Logic', ha='center', fontsize=14, fontweight='bold', color='#92400e')
ax.text(107.5, 44, r'$Z = Q_1$', ha='center', fontsize=12, color='#92400e')

ax.annotate('', xy=(10, 47), xytext=(0, 47), arrowprops=dict(arrowstyle='->', color=wire_color, lw=2))
ax.text(2, 50, 'X', ha='center', fontsize=16, fontweight='bold')

ax.plot([40, 55], [55, 59], color=wire_color, lw=1.5)
ax.plot([40, 55], [40, 34], color=wire_color, lw=1.5)
ax.plot([75, 95], [59, 50], color=wire_color, lw=1.5)

ax.plot([75, 85, 85, 5, 5, 10], [59, 59, 75, 75, 52, 52], color='#8b5cf6', lw=1.5, linestyle='--')
ax.plot([75, 88, 88, 3, 3, 10], [34, 34, 78, 78, 42, 42], color='#8b5cf6', lw=1.5, linestyle='--')

ax.annotate('', xy=(135, 50), xytext=(120, 50), arrowprops=dict(arrowstyle='->', color=wire_color, lw=2))
ax.text(132, 53, 'Z', ha='center', fontsize=16, fontweight='bold')

plt.show()

### The General FSM Architecture

Every synchronous FSM follows this pattern:

1. **State Register**: Flip-flops store the current state (Q values)
2. **Next-State Logic**: Combinational circuit computing D inputs from current state and inputs
3. **Output Logic**: Combinational circuit computing outputs
   - Moore: Output depends only on Q (current state)
   - Mealy: Output depends on Q and inputs
4. **Clock**: Synchronizes state transitions

---

## Complete Example: Traffic Light Controller

Let's design a simplified traffic light controller for a single intersection. The light cycles through:
- **Green** (30 time units)
- **Yellow** (5 time units)  
- **Red** (30 time units)

For simplicity, we'll use a `timer_done` input signal that goes high when the current phase should end.

In [ ]:
from graphviz import Digraph
from IPython.display import display, SVG

dot = Digraph('traffic_light')
dot.attr(rankdir='LR')
dot.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='11', width='0.9')
dot.attr('edge', fontname='Arial', fontsize='9')

dot.node('GREEN', 'GREEN\nR=0,Y=0,G=1', fillcolor='#22c55e', fontcolor='white')
dot.node('YELLOW', 'YELLOW\nR=0,Y=1,G=0', fillcolor='#eab308', fontcolor='#422006')
dot.node('RED', 'RED\nR=1,Y=0,G=0', fillcolor='#ef4444', fontcolor='white')

dot.node('reset', '', shape='point', width='0')
dot.edge('reset', 'GREEN', label='Reset')
dot.edge('GREEN', 'YELLOW', label='T=1', penwidth='2')
dot.edge('YELLOW', 'RED', label='T=1', penwidth='2')
dot.edge('RED', 'GREEN', label='T=1', penwidth='2')
dot.edge('GREEN', 'GREEN', label='T=0', color='#9ca3af', fontcolor='#6b7280')
dot.edge('YELLOW', 'YELLOW', label='T=0', color='#9ca3af', fontcolor='#6b7280')
dot.edge('RED', 'RED', label='T=0', color='#9ca3af', fontcolor='#6b7280')

display(SVG(dot.pipe(format='svg')))

### State Table for Traffic Light Controller (Binary Encoding)

We'll use **binary encoding** for the three states:

| State | Q1 Q0 |
|-------|-------|
| GREEN  | 0  0  |
| YELLOW | 0  1  |
| RED    | 1  0  |

**Why binary encoding?** For 3 states, we need only ⌈log₂(3)⌉ = 2 flip-flops. Binary encoding is efficient for this small state space.

The complete state table with encoded states:

| Current State | Q1 Q0 | timer_done | R | Y | G | Next State | D1 D0 |
|--------------|-------|------------|---|---|---|------------|-------|
| GREEN        | 0  0  | 0          | 0 | 0 | 1 | GREEN      | 0  0  |
| GREEN        | 0  0  | 1          | 0 | 0 | 1 | YELLOW     | 0  1  |
| YELLOW       | 0  1  | 0          | 0 | 1 | 0 | YELLOW     | 0  1  |
| YELLOW       | 0  1  | 1          | 0 | 1 | 0 | RED        | 1  0  |
| RED          | 1  0  | 0          | 1 | 0 | 0 | RED        | 1  0  |
| RED          | 1  0  | 1          | 1 | 0 | 0 | GREEN      | 0  0  |

**How to read this table:**

- **Current State & Q1,Q0**: The state we're in and its binary encoding
- **timer_done**: Input signal (1 = time to advance to next light)
- **R, Y, G**: Output signals *while in the current state* (Moore machine - depends only on current state encoding)
- **Next State & D1,D0**: Where we transition after the clock edge, and the flip-flop input values needed

**Note:** This reads naturally left-to-right: "In [current state] with [input], outputs are [R,Y,G], then transition to [next state]"

### Boolean Equations

From the state table:

$$D_1 = Q_0 \cdot T + Q_1 \cdot \overline{T}$$

$$D_0 = \overline{Q_1} \cdot \overline{Q_0} \cdot T + Q_0 \cdot \overline{T}$$

$$R = Q_1, \quad Y = Q_0, \quad G = \overline{Q_1} \cdot \overline{Q_0}$$

where $T$ = timer_done.

---

## Timing Analysis

Understanding FSM timing is critical for correct operation. Let's trace through our sequence detector.

**About the input sequence:** The X pattern `[0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0]` was carefully chosen to demonstrate:

1. **First detection (cycles 1-3)**: Two consecutive 1s → S0→S1→S2, Z becomes 1
2. **Reset mid-sequence (cycle 3-4)**: X=0 breaks the streak → back to S0
3. **Second detection (cycles 4-7)**: Three consecutive 1s → detects at cycle 6, stays in S2 for cycle 7
4. **Double reset (cycles 7-8)**: Two 0s in a row → confirms we're back at S0
5. **Third detection (cycles 9-11)**: Another pair of 1s → detection at cycle 11

**Why this pattern is instructive:**
- Shows multiple detections in one trace
- Demonstrates that Z=1 persists while X=1 continues (cycles 6-7)
- Shows immediate reset to S0 on X=0 from any state
- The 0s at different points show the FSM properly restarting

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)
logging.getLogger('matplotlib.font_manager').disabled = True

import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update({"text.usetex": False, "mathtext.fontset": "cm"})
fig, axes = plt.subplots(4, 1, figsize=(14, 8), sharex=True)
t = np.arange(0, 12, 0.01)
clk = np.zeros_like(t)
for i in range(12): clk[(t >= i) & (t < i + 0.5)] = 1
x_values = [0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0]
x_signal = np.zeros_like(t)
for i, val in enumerate(x_values): x_signal[(t >= i) & (t < i + 1)] = val
state_values = [0, 0, 1, 2, 0, 1, 2, 2, 0, 0, 1, 2]
state_signal = np.zeros_like(t)
for i, val in enumerate(state_values): state_signal[(t >= i) & (t < i + 1)] = val
z_signal = np.zeros_like(t)
for i, val in enumerate(state_values): z_signal[(t >= i) & (t < i + 1)] = 1 if val == 2 else 0

axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre'); axes[0].set_ylabel('CLK')
axes[1].plot(t, x_signal, color='#10b981', lw=2, drawstyle='steps-pre'); axes[1].set_ylabel('X (Input)')
axes[2].plot(t, state_signal, color='#374151', lw=2, drawstyle='steps-pre'); axes[2].set_ylabel('State')
axes[3].plot(t, z_signal, color='#ef4444', lw=2, drawstyle='steps-pre'); axes[3].set_ylabel('Z (Output)')

plt.tight_layout(); plt.show()

**Reading the timing diagram:**

1. State changes occur at the **rising edge** of the clock
2. The new state depends on the **current state** and **input at the clock edge**
3. In a Moore machine, output Z changes only when the state changes
4. Detection (Z=1) occurs when we reach state S2, which happens after seeing two consecutive 1s

---

## Common FSM Design Patterns

### Pattern 1: Sequence Detector
Detects a specific bit pattern in a stream. We built one above.

### Pattern 2: Counter
Cycles through states in order. Each state represents a count value.

### Pattern 3: Controller
Orchestrates multi-step operations. States represent phases of an operation (like the traffic light).

### Pattern 4: Arbiter
Manages access to shared resources. States track which requester has access.

In [ ]:
from graphviz import Digraph
from IPython.display import display, HTML

counter = Digraph('counter')
counter.attr(label='3-Bit Counter', labelloc='t')
counter.attr('node', shape='circle', style='filled', fillcolor='#3b82f6', fontcolor='white')
for i in range(8): counter.node(f'C{i}', f'{i}\n{i:03b}')
for i in range(8): counter.edge(f'C{i}', f'C{(i+1)%8}')

arbiter = Digraph('arbiter')
arbiter.attr(label='Simple Arbiter', labelloc='t')
arbiter.attr('node', shape='circle', style='filled', fillcolor='#6b7280', fontcolor='white')
arbiter.node('IDLE', 'IDLE')
arbiter.node('GA', 'GRANT A', fillcolor='#3b82f6')
arbiter.node('GB', 'GRANT B', fillcolor='#10b981')
arbiter.edge('IDLE', 'GA', label='reqA=1')
arbiter.edge('IDLE', 'GB', label='reqA=0,reqB=1')
arbiter.edge('GA', 'IDLE', label='done'); arbiter.edge('GB', 'IDLE', label='done')

display(HTML(f'<div style="display:flex">{counter.pipe(format="svg").decode("utf-8")}{arbiter.pipe(format="svg").decode("utf-8")}</div>'))

---

## Key Takeaways

1. **FSMs are the foundation** of sequential digital design — nearly every digital system contains FSMs

2. **Moore vs Mealy** is a fundamental design choice:
   - Moore: Simpler timing, outputs change only on clock edges
   - Mealy: Potentially fewer states, faster response to inputs

3. **State encoding** affects implementation complexity:
   - Binary minimizes flip-flops
   - One-hot simplifies combinational logic

4. **The general architecture** is always:
   - State register (flip-flops)
   - Next-state combinational logic
   - Output combinational logic
   - Clock: Synchronizes state transitions

5. **Design process**:
   1. Draw state diagram from requirements
   2. Create state table
   3. Choose state encoding
   4. Derive Boolean equations
   5. Implement with gates and flip-flops (or HDL)

---

*FSMs are everywhere in digital design — from simple button debouncers to complex CPU control units. Master them, and you have a powerful tool for any sequential logic problem.*